# Results

In [24]:
import re
import ast
import json
import math
import functools
import operator
import collections
import numpy as np
import pandas as pd
from pathlib import Path
from collections import defaultdict
from metrics import recall, mrr, ndcg, score

In [25]:
def clean_string(string):
    return re.sub(r'\W+','', string).lower() 

def compute_results(trues, preds_annots):
    data = {}
    for k, preds in preds_annots.items():
        data[k] = score(trues, preds, [recall, ndcg, mrr], k=20)
    df = pd.DataFrame(data).transpose()
    df.loc['Mean'] = df.mean()
    return df.round(1)

# Comparison of basic approaches and search engine on ACL Anthology dataset

In [26]:
reading_lists = pd.read_csv("../reading_lists.csv")
reading_lists['reading_list'] = reading_lists['reading_list'].apply(ast.literal_eval)

preds_semantic_scholar_annot1 = { id_:[clean_string(ref["title"]) for ref in references] for id_, references in json.loads(Path('preds_acl_collection/preds/semantic_scholar/preds_annot1.json').read_text()).items()}
preds_semantic_scholar_annot2 = { id_:[clean_string(ref["title"]) for ref in references] for id_, references in json.loads(Path('preds_acl_collection/preds/semantic_scholar/preds_annot2.json').read_text()).items()}
preds_semantic_scholar_annot3 = { id_:[clean_string(ref["title"]) for ref in references] for id_, references in json.loads(Path('preds_acl_collection/preds/semantic_scholar/preds_annot3.json').read_text()).items()}

preds_bm25_annot1 = { id_:[clean_string(ref["title"]) for ref in references] for id_, references in  json.loads(Path('preds_acl_collection/preds/bm25/preds_annot1.json').read_text()).items()}
preds_bm25_annot2 = { id_:[clean_string(ref["title"]) for ref in references] for id_, references in  json.loads(Path('preds_acl_collection/preds/bm25/preds_annot2.json').read_text()).items()}
preds_bm25_annot3 = { id_:[clean_string(ref["title"]) for ref in references] for id_, references in  json.loads(Path('preds_acl_collection/preds/bm25/preds_annot3.json').read_text()).items()}

preds_specterv2_annot1 = { id_:[clean_string(ref["title"]) for ref in references] for id_, references in  json.loads(Path('preds_acl_collection/preds/specterv2/preds_annot1.json').read_text()).items()}
preds_specterv2_annot2 = { id_:[clean_string(ref["title"]) for ref in references] for id_, references in  json.loads(Path('preds_acl_collection/preds/specterv2/preds_annot2.json').read_text()).items()}
preds_specterv2_annot3 = { id_:[clean_string(ref["title"]) for ref in references] for id_, references in  json.loads(Path('preds_acl_collection/preds/specterv2/preds_annot3.json').read_text()).items()}

trues = { id_:[clean_string(ref["title"]) for ref in references if ref["acl_id"]] for id_, references in zip(reading_lists["id"], reading_lists["reading_list"]) }

preds = {
    "PhD Student": [preds_semantic_scholar_annot1, preds_bm25_annot1, preds_specterv2_annot1], 
    "Expert-Level": [preds_semantic_scholar_annot2, preds_bm25_annot2, preds_specterv2_annot2],
    "PhD-Level": [preds_semantic_scholar_annot3, preds_bm25_annot3, preds_specterv2_annot3]
}

In [33]:
df1 = compute_results(trues, {"PhD Student":preds_semantic_scholar_annot1, "Expert-Level":preds_semantic_scholar_annot2, "PhD-Level":preds_semantic_scholar_annot3})
df2 = compute_results(trues, {"PhD Student":preds_bm25_annot1, "Expert-Level":preds_bm25_annot2, "PhD-Level":preds_bm25_annot3})
df3 = compute_results(trues, {"PhD Student":preds_specterv2_annot1, "Expert-Level":preds_specterv2_annot2, "PhD-Level":preds_specterv2_annot3})
df_engines = pd.concat([df1, df2, df3], axis=1, keys=["Semantic Scholar", "BM25", "SPECTER2"]); df_engines

#print(df_engines.to_latex(float_format="{:0.1f}".format))

Semantic Scholar              BM25           SPECTER2          
                       recall ndcg   mrr recall ndcg  mrr   recall ndcg  mrr
PhD Student               8.4  5.5   7.6    8.9  5.7  8.8      5.9  3.1  3.3
Expert-Level              9.5  7.2  11.0   11.2  6.6  9.2      8.2  4.7  5.8
PhD-Level                 6.7  5.5   8.2    9.2  5.5  6.3      5.7  3.4  4.6
Mean                      8.2  6.1   9.0    9.7  5.9  8.1      6.6  3.7  4.6

## Matches of correct prediction

In [28]:
def match(trues, preds1, preds2):
    match = 0
    size = 0
    for k, trues_v in trues.items():
        if trues_v != []:
            match1 = set(dict.fromkeys(trues_v)) & set(dict.fromkeys(preds1[k]))
            match2 = set(dict.fromkeys(trues_v)) & set(dict.fromkeys(preds2[k]))
            match+=len(match1 & match2)
            size+=1
    return match/size

In [29]:
(match(trues, preds_bm25_annot1, preds_specterv2_annot1) +\
match(trues, preds_bm25_annot2, preds_specterv2_annot2) +\
match(trues, preds_bm25_annot3, preds_specterv2_annot3)) /3

0.1050228310502283